# Scripts de Qualification des données Geod'Air 

## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Functions

In [12]:
# lire les données d'un poluant à partir des fichiers csv de 2013 jusqu'au 2023
def LoadData(path):
    files = os.listdir(path)
    data = pd.DataFrame()
    for file in files : 
        file_df = pd.read_csv(path+ file, sep=';')
        data = pd.concat([data, file_df]).reset_index(drop=True)
    return data

# retourner les données d'un polluant pour une organisme donnée
def data_org(data, organisme):
    data_organisme = data[data['Organisme']==organisme]
    return data_organisme

# retourner te taux des données manquantes pour un polluant sur une periode 
def proportion_jours_manquants(data_organisme):
    date_début = data_organisme.index[0]
    date_fin = data_organisme.index[-1]
    jours_manquants = pd.date_range(start = date_début, end = date_fin).difference(data_organisme.index)
    proportion = len(jours_manquants)/len(data_organisme)
    print("***")
    print(f" L'organisme {data_organisme['Organisme'].unique()[0]} a prix les mesures du {date_début} jusqu'au {date_fin}")
    print(f" Entre ces deux dates {len(jours_manquants)} jours manquent soit un taux de  {proportion: .4f}")

    return jours_manquants, proportion, date_début, date_fin

In [13]:
# lire les données du polluant PM25
Data = LoadData('./PM25/')

In [14]:
Data.head()

,Date de début,Date de fin,Organisme,code zas,Zas,code site,nom site,type d'implantation,Polluant,type d'influence,...,valeur,valeur brute,unité de mesure,taux de saisie,couverture temporelle,couverture de données,code qualité,validité,Latitude,Longitude
0,2013/01/01 00:00:00,2013/01/01 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01011,Metz-Centre,Urbaine,PM2.5,Fond,...,13.0,12.675000,µg-m3,100.0,100,100.0,A,1,49.119442,6.180833
1,2013/01/01 00:00:00,2013/01/01 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01020,Thionville-Centre,Urbaine,PM2.5,Fond,...,6.1,6.125000,µg-m3,100.0,100,100.0,A,1,49.358337,6.156942
2,2013/01/01 00:00:00,2013/01/01 23:59:59,ATMO SUD,FR02N30,PROVENCE-ALPES-COTE-D-AZUR-ZI,FR02020,Rognac les Brets,Périurbaine,PM2.5,Fond,...,14.0,13.654167,µg-m3,100.0,100,100.0,R,1,43.510802,5.220046
3,2013/01/01 00:00:00,2013/01/01 23:59:59,ATMO SUD,FR03A02,PROVENCE-ALPES-COTE-D-AZUR-MARSEILLE,FR03006,MARSEILLE RABATAU,Urbaine,PM2.5,Trafic,...,17.0,16.650000,µg-m3,100.0,100,100.0,A,1,43.276451,5.397360
4,2013/01/01 00:00:00,2013/01/01 23:59:59,ATMO SUD,FR03A02,PROVENCE-ALPES-COTE-D-AZUR-MARSEILLE,FR03029,AIX CENTRE ECOLE ART,Urbaine,PM2.5,Fond,...,22.0,22.254167,µg-m3,100.0,100,100.0,A,1,43.530285,5.440912


## Exctraction d'information

In [15]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603703 entries, 0 to 603702
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Date de début          603703 non-null  object 
 1   Date de fin            603703 non-null  object 
 2   Organisme              603703 non-null  object 
 3   code zas               603703 non-null  object 
 4   Zas                    603703 non-null  object 
 5   code site              603703 non-null  object 
 6   nom site               603703 non-null  object 
 7   type d'implantation    603703 non-null  object 
 8   Polluant               603703 non-null  object 
 9   type d'influence       603703 non-null  object 
 10  Réglementaire          603703 non-null  object 
 11  type d'évaluation      603703 non-null  object 
 12  type de valeur         603703 non-null  object 
 13  valeur                 603703 non-null  float64
 14  valeur brute           603703 non-nu

In [16]:
Data.describe()

,valeur,valeur brute,taux de saisie,couverture temporelle,couverture de données,validité,Latitude,Longitude
count,603703.000000,603703.000000,603703.000000,603703.000000,603703.000000,603703.000000,603703.000000,603703.000000
mean,14.584705,14.585179,94.430520,99.999861,94.430520,0.877519,44.312516,2.744882
std,153.585409,153.584913,19.719171,0.082991,19.719171,0.479542,12.034001,13.218469
min,-2.000000,-1.981818,0.000000,38.000000,0.000000,-1.000000,-21.342915,-63.081481
25%,5.100000,5.141667,100.000000,100.000000,100.000000,1.000000,44.548647,0.758833
50%,8.200000,8.183333,100.000000,100.000000,100.000000,1.000000,46.942941,3.084158
75%,13.000000,13.145833,100.000000,100.000000,100.000000,1.000000,48.838615,5.360119
max,10000.000000,9999.900000,100.000000,100.000000,100.000000,1.000000,51.048580,55.627930


In [19]:
Data['Date de début'] = pd.to_datetime(Data['Date de début'])
Data['Date de fin'] = pd.to_datetime(Data['Date de fin'])

In [20]:
Data

,Date de début,Date de fin,Organisme,code zas,Zas,code site,nom site,type d'implantation,Polluant,type d'influence,...,valeur,valeur brute,unité de mesure,taux de saisie,couverture temporelle,couverture de données,code qualité,validité,Latitude,Longitude
0,2013-01-01,2013-01-01 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01011,Metz-Centre,Urbaine,PM2.5,Fond,...,13.0,12.675000,µg-m3,100.0,100,100.0,A,1,49.119442,6.180833
1,2013-01-01,2013-01-01 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01020,Thionville-Centre,Urbaine,PM2.5,Fond,...,6.1,6.125000,µg-m3,100.0,100,100.0,A,1,49.358337,6.156942
2,2013-01-01,2013-01-01 23:59:59,ATMO SUD,FR02N30,PROVENCE-ALPES-COTE-D-AZUR-ZI,FR02020,Rognac les Brets,Périurbaine,PM2.5,Fond,...,14.0,13.654167,µg-m3,100.0,100,100.0,R,1,43.510802,5.220046
3,2013-01-01,2013-01-01 23:59:59,ATMO SUD,FR03A02,PROVENCE-ALPES-COTE-D-AZUR-MARSEILLE,FR03006,MARSEILLE RABATAU,Urbaine,PM2.5,Trafic,...,17.0,16.650000,µg-m3,100.0,100,100.0,A,1,43.276451,5.397360
4,2013-01-01,2013-01-01 23:59:59,ATMO SUD,FR03A02,PROVENCE-ALPES-COTE-D-AZUR-MARSEILLE,FR03029,AIX CENTRE ECOLE ART,Urbaine,PM2.5,Fond,...,22.0,22.254167,µg-m3,100.0,100,100.0,A,1,43.530285,5.440912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603698,2023-05-09,2023-05-09 23:59:59,ATMO REUNION,FR04ZAR02,ZAR VOLCAN,FR38017,Grand Coude,Rurale régionale,PM2.5,Fond,...,4.3,4.291304,µg-m3,96.0,100,96.0,A,1,-21.294330,55.627930
603699,2023-05-09,2023-05-09 23:59:59,ATMO REUNION,FR04ZAR01,ZAR SAINT-DENIS,FR38020,Plateau Caillou,Urbaine,PM2.5,Fond,...,3.2,3.152174,µg-m3,96.0,100,96.0,A,1,-21.009739,55.272675
603700,2023-05-09,2023-05-09 23:59:59,ATMO REUNION,FR04ZAR02,ZAR VOLCAN,FR38023,BD BANKS,Urbaine,PM2.5,Trafic,...,0.0,0.000000,µg-m3,0.0,100,0.0,N,-1,-21.331703,55.478949
603701,2023-05-09,2023-05-09 23:59:59,ATMO REUNION,FR04ZRE01,ZR LA REUNION,FR38024,RDT SAINT-LEU,Rurale près des villes,PM2.5,Trafic,...,4.0,4.037500,µg-m3,100.0,100,100.0,A,1,-21.213212,55.302844


## Profondeur Historique

In [21]:
Data['Date de début'].min()

Timestamp('2013-01-01 00:00:00')

## Granularité Temporelle
La granularité temporelle des données du polluant PM25 est de l'ordre d'1 jour.

In [23]:
(Data['Date de début'] - Data['Date de fin']).min(), (Data['Date de début'] - Data['Date de fin']).max()

(Timedelta('-1 days +00:00:01'), Timedelta('-1 days +06:00:00'))

## Couverture Géographique, Granularité spatiale, Régions
Les données du polluant PM25 sont prises sur les 18 régions de la France métropolitaine et outre mer.

In [34]:
# France métropolitaine et outre mer, 18 régions 
Data.Zas.unique()

array(['LORRAINE-METZ', 'PROVENCE-ALPES-COTE-D-AZUR-ZI',
       'PROVENCE-ALPES-COTE-D-AZUR-MARSEILLE',
       'PROVENCE-ALPES-COTE-D-AZUR-TOULON',
       'PROVENCE-ALPES-COTE-D-AZUR-AVIGNON', 'ILE-DE-FRANCE-PARIS',
       'ILE-DE-FRANCE-ZR', 'HAUTE-NORMANDIE-ZI', 'HAUTE-NORMANDIE-ZR',
       'NORD-PAS-DE-CALAIS-DOUAI-BETHUNE-VALENCIENNES',
       'NORD-PAS-DE-CALAIS-ZR', 'AUVERGNE-CLERMONT',
       'LANGUEDOC-ROUSSILLON-MONTPELLIER', 'LANGUEDOC-ROUSSILLON-ZUR',
       'POITOU-CHARENTES-ZUR', 'NORD-PAS-DE-CALAIS-ZUR',
       'NORD-PAS-DE-CALAIS-LILLE', 'MIDI-PYRENEES-ZR',
       'MIDI-PYRENEES-TOULOUSE', 'CHAMPAGNE-ARDENNE-ZR',
       'CHAMPAGNE-ARDENNE-ZUR', 'RHONE-ALPES-GRENOBLE',
       'ALSACE-STRASBOURG', 'ALSACE-MULHOUSE', 'PICARDIE-ZUR',
       'BRETAGNE-RENNES', 'BRETAGNE-ZR', 'BRETAGNE-ZUR',
       'RHONE-ALPES-LYON', 'BASSE-NORMANDIE-ZUR', 'BASSE-NORMANDIE-ZR',
       'PAYS DE LA LOIRE-NANTES', 'PAYS DE LA LOIRE-ZUR',
       'PAYS DE LA LOIRE-ZR', 'PROVENCE-ALPES-COTE-D-AZUR-

## Présence des coordonnées GPS des stations

In [25]:
Data[['Latitude', 'Longitude']]

,Latitude,Longitude
0,49.119442,6.180833
1,49.358337,6.156942
2,43.510802,5.220046
3,43.276451,5.397360
4,43.530285,5.440912
...,...,...
603698,-21.294330,55.627930
603699,-21.009739,55.272675
603700,-21.331703,55.478949
603701,-21.213212,55.302844


## Indicateur sur la typologie des stations
Il y a 3 typologie des stations : Fond, Trafic et Indutrielle

In [26]:
Data["type d'influence"].unique()

array(['Fond', 'Trafic', 'Industrielle'], dtype=object)

## Type d'implantation
Il y a 5 types d'implantations : Urbaine, Périuraine, Rurale régionale, Rurale nationale et Rurale près des villes

In [27]:
Data["type d'implantation"].unique()

array(['Urbaine', 'Périurbaine', 'Rurale régionale', 'Rurale nationale',
       'Rurale près des villes'], dtype=object)

## Taux de données manquantes

In [28]:
Data.isnull().sum()

Date de début            0
Date de fin              0
Organisme                0
code zas                 0
Zas                      0
code site                0
nom site                 0
type d'implantation      0
Polluant                 0
type d'influence         0
Réglementaire            0
type d'évaluation        0
type de valeur           0
valeur                   0
valeur brute             0
unité de mesure          0
taux de saisie           0
couverture temporelle    0
couverture de données    0
code qualité             0
validité                 0
Latitude                 0
Longitude                0
dtype: int64

## Présence d’indicateurs sur la qualité des mesures
### Taux de saisie

In [29]:
Data['taux de saisie'].mean()

94.43051964293701

 ### Code Qualité de mesure : 

In [30]:
Data['code qualité'].value_counts()/len(Data['code qualité'])*100

A    81.599727
R    12.276235
N     6.124038
Name: code qualité, dtype: float64

### Validité

In [31]:
Data['validité'].value_counts()/len(Data['validité'])*100

 1    93.875962
-1     6.124038
Name: validité, dtype: float64

### Valeurs manquantes par Région

In [32]:
Data.set_index('Date de début', inplace=True)
Data.head()

,Date de fin,Organisme,code zas,Zas,code site,nom site,type d'implantation,Polluant,type d'influence,Réglementaire,...,valeur,valeur brute,unité de mesure,taux de saisie,couverture temporelle,couverture de données,code qualité,validité,Latitude,Longitude
Date de début,,,,,,,,,,,,,,,,,,,,,
2013-01-01,2013-01-01 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01011,Metz-Centre,Urbaine,PM2.5,Fond,Oui,...,13.0,12.675000,µg-m3,100.0,100,100.0,A,1,49.119442,6.180833
2013-01-01,2013-01-01 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01020,Thionville-Centre,Urbaine,PM2.5,Fond,Oui,...,6.1,6.125000,µg-m3,100.0,100,100.0,A,1,49.358337,6.156942
2013-01-01,2013-01-01 23:59:59,ATMO SUD,FR02N30,PROVENCE-ALPES-COTE-D-AZUR-ZI,FR02020,Rognac les Brets,Périurbaine,PM2.5,Fond,Oui,...,14.0,13.654167,µg-m3,100.0,100,100.0,R,1,43.510802,5.220046
2013-01-01,2013-01-01 23:59:59,ATMO SUD,FR03A02,PROVENCE-ALPES-COTE-D-AZUR-MARSEILLE,FR03006,MARSEILLE RABATAU,Urbaine,PM2.5,Trafic,Oui,...,17.0,16.650000,µg-m3,100.0,100,100.0,A,1,43.276451,5.397360
2013-01-01,2013-01-01 23:59:59,ATMO SUD,FR03A02,PROVENCE-ALPES-COTE-D-AZUR-MARSEILLE,FR03029,AIX CENTRE ECOLE ART,Urbaine,PM2.5,Fond,Oui,...,22.0,22.254167,µg-m3,100.0,100,100.0,A,1,43.530285,5.440912


In [33]:
for org_val in Data['Organisme'].unique():
    data_organisme = data_org(Data, org_val)
    jours_manquants, proportion, date_début, date_fin = proportion_jours_manquants(data_organisme)

***
 L'organisme ATMO GRAND EST a prix les mesures du 2013-01-01 00:00:00 jusqu'au 2023-05-09 00:00:00
 Entre ces deux dates 0 jours manquent soit un taux de   0.0000
***
 L'organisme ATMO SUD a prix les mesures du 2013-01-01 00:00:00 jusqu'au 2023-05-09 00:00:00
 Entre ces deux dates 0 jours manquent soit un taux de   0.0000
***
 L'organisme AIRPARIF a prix les mesures du 2013-01-01 00:00:00 jusqu'au 2023-05-09 00:00:00
 Entre ces deux dates 0 jours manquent soit un taux de   0.0000
***
 L'organisme ATMO NORMANDIE a prix les mesures du 2013-01-01 00:00:00 jusqu'au 2023-05-09 00:00:00
 Entre ces deux dates 0 jours manquent soit un taux de   0.0000
***
 L'organisme ATMO HAUTS DE FRANCE a prix les mesures du 2013-01-01 00:00:00 jusqu'au 2023-05-09 00:00:00
 Entre ces deux dates 0 jours manquent soit un taux de   0.0000
***
 L'organisme ATMO AUVERGNE-RHÔNE-ALPES a prix les mesures du 2013-01-01 00:00:00 jusqu'au 2023-05-09 00:00:00
 Entre ces deux dates 0 jours manquent soit un taux de   